In [ ]:
import mercury as mr
import csv

show_code = mr.Checkbox(value=False, label="Show source code")
app = mr.App(
    title="Loan House Calculation",
    description="Actuarial Mathematics",
    show_code=show_code.value,
    continuous_update=False
)
_ = mr.Note(text="---")

age = mr.Numeric(label="Age (Years)", value=22, min=18, max=100)
amount = mr.Numeric(label="Loan Amount (EUR)", min=0, value=150000, max=999999999999)
interest_rate = mr.Numeric(label="Yearly Interest Rate (%)", value=2.5, min=0.0, max=100.0, step=0.1)
maturity = mr.Numeric(label="Maturity (Years)", value=25, min=1, max=50)
mortality_table = mr.Select(value = "TD88-90 (France)", choices = ["TD88-90 (France)"], label = "Select Mortality Table")

_ = mr.Note(text="---")
_ = mr.Note("Please fill out the inputs above and click the green _Run_ button to calculate the result.")

In [128]:
# Input validation
if age.value + maturity.value > 100:
    print("With these values no house loan is available.")
    mr.Stop()

In [127]:


# Load mortality table based on selection
if mortality_table.value == "TD88-90 (France)":
    with open("TD88-90.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        mortality_data = {int(row[0]): int(row[1]) for row in reader}

print(f"Successfully loaded {len(mortality_data)} entries from {mortality_table.value} mortality table.")

# Basic calculations
number_of_payments = int(maturity.value * 12) # input is in years
periodic_rate = interest_rate.value / 12 / 100 # input is yearly + should be divided by 100
monthly_reimbursed_amount = (amount.value * periodic_rate) / (1 - (1 + periodic_rate) ** -number_of_payments)

print("----")
print(f"Number of payments: {number_of_payments}")
print(f"Periodic rate: {periodic_rate:.6f}" + "%")
print(f"Total amount: {amount.value:.0f}")
print(f"Monthly reimbursed amount: {monthly_reimbursed_amount:.6f} EUR")

# Calculate for each month
remaining_amount = []
interest = []
amortization = []
tolerance = 1e-7

for i in range(0, number_of_payments + 1):
    # Special cases on the 0 month
    if i == 0:
        remaining_amount.append(amount.value)
        interest.append(0)
        amortization.append(0)
        continue

    # Remaining amount (end of the year) Ck
    # Using tolerance for the last element (not perfect zero)
    calculated_value = (1 + periodic_rate) * remaining_amount[-1] - monthly_reimbursed_amount
    remaining_amount.append(0 if calculated_value < tolerance else calculated_value)

    # Interest Ik
    interest.append(remaining_amount[i-1] * periodic_rate)
    # Amortization Ak
    amortization.append(monthly_reimbursed_amount - interest[i])

total_reimbursement_amount = sum(amortization)
cost_of_loan = sum(interest)

print("----")
print(f"Total reimbursement amount: {total_reimbursement_amount:.2f}")
print(f"Cost of loan: {cost_of_loan:.2f}")

# Details calculation
k = 0
discounted_amount_in_risks = []

for i in range(1, len(remaining_amount)):
    if i % 12 == 0:
        benefit_akprime = remaining_amount[i]
        discount_factor = (1 / (1 + interest_rate.value / 100)) ** (k+1)
        probability = (mortality_data[age.value+k] - mortality_data[age.value+k+1]) / mortality_data[age.value]
        discounted_amount_in_risk = benefit_akprime * discount_factor * probability
        discounted_amount_in_risks.append(discounted_amount_in_risk)
        k += 1
total_discounted_amount_in_risk = sum(discounted_amount_in_risks)
single_premium = total_discounted_amount_in_risk

print(f"Total discounted amount in risks: {total_discounted_amount_in_risk:.2f}")
print(f"Single premium (yearly payment): {single_premium:.2f}")

Successfully loaded 107 entries from TD88-90 mortality table.
----
Number of payments: 300
Periodic rate: 0.002083%
Total amount: 150000
Monthly reimbursed amount: 672.925101 EUR
----
Total reimbursement amount: 150000.00
Cost of loan: 51877.53
Total discounted amount in risks: 2914.63
Single premium (yearly payment): 2914.63
